Uses USGC Earthquake Catalog API (https://earthquake.usgs.gov/fdsnws/event/1/#parameters)

In [1]:
import pandas as pd
import folium
from folium.plugins import MousePosition
import branca
from branca.element import Template, MacroElement
import requests
import ipywidgets as widgets
from IPython.display import display, clear_output

In [2]:
def get_earthquake_data(params):
    start_time = f'now-{params["start_time"]}days'
    min_magnitude = params['min_magnitude']
    max_magnitude = params['max_magnitude']
    min_depth = params['min_depth']
    max_depth = params['max_depth']

    use_circle_search = params['use_circle_search']
    circle_lat = params['circle_lat']
    circle_long = params['circle_long']
    circle_radius = params['circle_radius']
    
    req_url = f'https://earthquake.usgs.gov/fdsnws/event/1/query?format=geojson'
    req_url += f'&starttime={start_time}'
    req_url += f'&minmagnitude={min_magnitude}&maxmagnitude={max_magnitude}'
    req_url += f'&mindepth={min_depth}&maxdepth={max_depth}'
    if use_circle_search:
        if (circle_lat is not None) & (circle_long is not None) & (circle_radius is not None):
            req_url += f'&latitude={circle_lat}&longitude={circle_long}&maxradiuskm={circle_radius}'
        else:
            return None

    dataset = requests.get(req_url).json()
    if dataset['metadata']['count'] < 1:
        return None
    features = dataset['features']
    titles = []
    mags = []
    times = []
    lats = []
    longs = []
    depths = []
    urls = []

    for feature in features:
        titles.append(feature['properties']['title'])
        mags.append(feature['properties']['mag'])
        times.append(pd.to_datetime(feature['properties']['time'], unit='ms').strftime('%y/%m/%d %H:%M:%S'))
        lats.append(feature['geometry']['coordinates'][1])
        longs.append(feature['geometry']['coordinates'][0])
        depths.append(feature['geometry']['coordinates'][2])
        urls.append(feature['properties']['url'])

    df = pd.DataFrame({
        'title': titles,
        'magnitude': mags,
        'depth': depths,
        'date_time': times,
        'latitude': lats,
        'longitude': longs,
        'url': urls
    })
    return df

In [3]:
def get_earthquake_map(df):

    geo_json_url = 'https://raw.githubusercontent.com/fraxen/tectonicplates/master/GeoJSON/PB2002_boundaries.json'

    min_zoom = 2
    color_lst = ['purple', 'blue', 'green', 'yellow', 'orange', 'red']
    
    tile_graysale = folium.TileLayer(
        tiles = 'cartodb positron',
        attr = '© OpenStreetMap contributors © CARTO',
        name = 'Grayscale',
        overlay = False,
        control = True,
        min_zoom=min_zoom
       )
    
    m = folium.Map(location=[0, 0], tiles=tile_graysale, zoom_start=4, min_zoom=min_zoom, control_scale=True)

    folium.TileLayer(
        tiles = 'https://server.arcgisonline.com/ArcGIS/rest/services/World_Imagery/MapServer/tile/{z}/{y}/{x}',
        attr = 'Powered by Esri',
        name = 'Satellite',
        overlay = False,
        control = True,
        show=False,
        min_zoom=min_zoom
       ).add_to(m)
    
    folium.TileLayer(
        tiles="openstreetmap",
        attr='© OpenStreetMap contributors',
        name='Street',
        overlay=False,
        control=True,
        show=False,
        min_zoom=min_zoom
    ).add_to(m)


    #mc = MarkerCluster()

    for _, row in df.iterrows():
        if row.depth > 500:
            fcolor = 'red'
        elif row.depth > 300:
            fcolor = 'orange'
        elif row.depth > 150:
            fcolor = 'yellow'
        elif row.depth > 70:
            fcolor = 'green'
        elif row.depth > 35:
            fcolor = 'blue'
        else:
            fcolor = 'purple'
        
        if row.latitude > 0:
            ns_hem = 'N'
        elif row.latitude < 0:
            ns_hem = 'S'
        else:
            ns_hem = ''

        if row.longitude > 0:
            we_hem = 'E'
        elif row.longitude < 0:
            we_hem = 'W'
        else:
            we_hem = ''
        
        popup_html = f"""
            <div style="font-family: Arial;">
                <h3><a href={row.url} target="_top">{row.title}</a></h3>
                <font color="grey">Time:</font> {row.date_time} (UTC)<br>
                <font color="grey">Location:</font> {abs(row.latitude)}&deg{ns_hem} {abs(row.longitude)}&deg{we_hem}<br>
                <font color="grey">Depth:</font> {row.depth} km
            </div>
        """

        popup = folium.Popup(branca.element.IFrame(html=popup_html, width=300, height=150))
            
        folium.CircleMarker(
            location=[row.latitude, row.longitude],
            radius=round(row.magnitude),
            color='black',
            opacity=1.0,
            weight=1.0,
            fill_color=fcolor,
            fill_opacity=1.0,
            popup=popup
        ).add_to(m)

    #mc.add_to(map)

    folium.GeoJson(
        geo_json_url,
        name='Plate Boundaries',
        style_function=lambda feature: {
                "color": "red",
                "weight": 0.75,
                "dashArray": "5, 5",
            },
    ).add_to(m)

    folium.LayerControl().add_to(m)

    formatter = "function(num) {return L.Util.formatNum(num, 3) + ' &deg; ';};"
    MousePosition(
        lat_formatter=formatter,
        lng_formatter=formatter,
    ).add_to(m)

    # Create the legend template as an HTML element
    depth_legend_template = """
    {% macro html(this, kwargs) %}
    <div id='maplegend' class='maplegend' 
        style='position: absolute; z-index: 9999; background-color: rgba(255, 255, 255, 0.5);
        border-radius: 6px; padding: 10px; font-size: 10.5px; left: 5px; bottom: 120px;'>

    Depth<br><br>
    <div class="colormap-container">
        <div class="colormap"></div><br>
        <div class="number" style="top: -7px;">0</div>
        <div class="number" style="top: 2%;">35</div>
        <div class="number" style="top: 8%;">70</div>
        <div class="number" style="top: 17%;">150</div>
        <div class="number" style="top: 35%;">300</div>
        <div class="number" style="top: 60%;">500</div>
        <div class="number" style="top: 97%;">800</div>
    </div>

    <style type='text/css'>
    .colormap-container {
        position: relative;
        width: 30px;
        height: 200px;
        margin-right: 10px;
        float: left;
    }

    .colormap {
        width: 10px;
        height: 200px;
        background: linear-gradient(to bottom, 
            #a020f0 0%,
            #a020f0 4.3756%,
            #0000ff 4.375%,
            #0000ff 8.75%,
            #008000 8.75%,
            #008000 18.75%, 
            #ffff00 18.75%,
            #ffff00 37.5%,
            #ffa500 37.5%,
            #ffa500 62.5%,
            #ff0000 62.5%,
            #ff0000 100%
        );
    }

    /* Define the number divs */
    .number {
        position: absolute;
        bottom: 0;
        left: 70%;
        transform: translateX(-50%);
        font-size: 9px;
        color: #333;
    }


    </style>
    {% endmacro %}
    """

    # Add the legend to the map
    macro = MacroElement()
    macro._template = Template(depth_legend_template)
    m.get_root().add_child(macro)

    # Create the legend template as an HTML element
    mag_legend_template = """
    {% macro html(this, kwargs) %}
    <div id='maplegend' class='maplegend' 
        style='position: absolute; z-index: 9999; background-color: rgba(255, 255, 255, 0.5);
        border-radius: 6px; padding: 10px; font-size: 10.5px; left: 0px; bottom: -70px;'>

    <div class='mag-legend'>
        Magnitude<br>
        <svg xmlns="http://www.w3.org/2000/svg" width="180" height="40">
            <circle cx="10" cy="9" r="9" fill="gray" stroke="black"/>
            <circle cx="30" cy="9" r="8" fill="gray" stroke="black"/>
            <circle cx="50" cy="9" r="7" fill="gray" stroke="black"/>
            <circle cx="70" cy="9" r="6" fill="gray" stroke="black"/>
            <circle cx="90" cy="9" r="5" fill="gray" stroke="black"/>
            <circle cx="110" cy="9" r="4" fill="gray" stroke="black"/>
            <circle cx="130" cy="9" r="3" fill="gray" stroke="black"/>
            <circle cx="150" cy="9" r="2" fill="gray" stroke="black"/>
            <circle cx="170" cy="9" r="1" fill="gray" stroke="black"/>
        </svg>
        <div class="number-2" style="left: 10px;">9</div>
        <div class="number-2" style="left: 30px;">8</div>
        <div class="number-2" style="left: 50px;">7</div>
        <div class="number-2" style="left: 70px;">6</div>
        <div class="number-2" style="left: 90px;">5</div>
        <div class="number-2" style="left: 110px;">4</div>
        <div class="number-2" style="left: 130px;">3</div>
        <div class="number-2" style="left: 150px;">2</div>
        <div class="number-2" style="left: 170px;">1</div>
        
    </div>
    </div> 
    <style type='text/css'>
    .mag-legend {
        position: relative;
        width: 180px;
        height: 40px;
    }
    .number-2 {
        position: absolute;
        bottom: -10px;
        transform: translateX(-50%);
        font-size: 12px;
        color: #333;
    }
    </style>
    {% endmacro %}
    """


    # Add the legend to the map
    macro = MacroElement()
    macro._template = Template(mag_legend_template)
    m.get_root().add_child(macro)

    return m

In [4]:
def get_map(params):
    df = get_earthquake_data(params)
    if df is not None:
        map = get_earthquake_map(df)
        return map
    else:
        print('No earthquakes found! Please change selection options.')
        return None

In [8]:
mag_w = widgets.FloatRangeSlider(
    value=[3, 10],
    min=0,
    max=10,
    step=0.5,
    description='Magnitude range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

depth_w = widgets.IntRangeSlider(
    value=[0, 800],
    min=0,
    max=800,
    step=10,
    description='Depth range:',
    disabled=False,
    continuous_update=False,
    orientation='horizontal',
    readout=True,
    readout_format='d'
)

period_w = widgets.Dropdown(
    options=[('Past 7 Days', 7), ('Past 30 Days', 30)],
    value=7,
    description='Date & Time:',
    disabled=False,
)

w_circle_use = widgets.Checkbox(
    value=False,
    description='Use circle search',
    disabled=False,
    indent=False
)

w_circle_lat = widgets.BoundedFloatText(
    min=-90,
    max=90,
    description='Latitude:',
    disabled=False
)

w_circle_long = widgets.BoundedFloatText(
    min =-180,
    max=180,
    description='Longitude:',
    disabled=False
)

w_circle_radius = widgets.BoundedFloatText(
    min=0,
    max=20001.6,
    description='Radius (km):',
    disabled=False
)


button = widgets.Button(description="Show")
output = widgets.Output()

display(period_w)
display(mag_w)
display(depth_w)
display(w_circle_use)
display(w_circle_lat)
display(w_circle_long)
display(w_circle_radius)
display(button, output)

def on_button_clicked(b):
    data_params = {
        'start_time': str(period_w.value),
        'min_magnitude': mag_w.value[0],
        'max_magnitude': mag_w.value[1],
        'min_depth': depth_w.value[0],
        'max_depth': depth_w.value[1],
        'use_circle_search': w_circle_use.value,
        'circle_lat': w_circle_lat.value,
        'circle_long': w_circle_long.value,
        'circle_radius': w_circle_radius.value
    }
    with output:
        clear_output()
        map = get_map(data_params)
        if map is not None:
            display(map)

button.on_click(on_button_clicked)

Dropdown(description='Date & Time:', options=(('Past 7 Days', 7), ('Past 30 Days', 30)), value=7)

IntRangeSlider(value=(3, 10), continuous_update=False, description='Magnitude range:', max=10, step=0)

IntRangeSlider(value=(0, 800), continuous_update=False, description='Depth range:', max=800, step=10)

Checkbox(value=False, description='Use circle search', indent=False)

BoundedFloatText(value=0.0, description='Latitude:', max=90.0, min=-90.0)

BoundedFloatText(value=0.0, description='Longitude:', max=180.0, min=-180.0)

BoundedFloatText(value=0.0, description='Radius (km):', max=20001.6)

Button(description='Show', style=ButtonStyle())

Output()